### Deteksi Keaktifan Penggunaan Covwatch

In [2]:
"""
Graph details:
#1 Person-based (n = nPerson12*nBio8 = 96)
  Person 1
Block 1: time (x) vs HR (left y) vs label (right y)
Block 2: time (x) vs Sys (left y) vs label (right y)
...
Person 2
=====
#2 Label-based (n = nLabel5*nBio8 = 40)
Label 1: Time(x) vs Bio1 (Person 1-n :shape-color y)
Label 1: Time(x) vs Bio2 (Person 1-n :shape-color y)
...
Label 0.75: Time(x) vs Bio2 (Person 1-n :shape-color y)
...
====
#3 Biomarker based ()
For every biomarker, plot the personal data vs the label (Oo-xX)

Daily = ['ACTIVITY_TIME',  'CALORIES', 'DISTANCE',  'STEP', 'SLEEP',]  
I have much data with these columns:
["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp", "Code", "Time (WIB)", "Name", "Device Code"]
Say the file dir is A/B.csv

Firstly, merge the Name & Device Code.
Secondly, create graph with Python's Plotly for each people (1 people 1 graph for each biomarker), the x-axis is the Time WIB, with vertical time label, the newest time in the right side. The first y-axis in the left is the biomarker columns (the column mentioned before without code, time, name, & device code. Plot with thin dots). The second y-axis in the right is the label data plot (plot this with thin line), scale this y-axis into the height of the biomarker plot
Finally, export these interactive graphs into html or so, so I could view it outside & give to other, but yet still interactive. Name it same as the source filename (not source dir).

====
# 4 : The Dropdown
- 1 graph
- Dropdowns: biomarker, 
- Checkbox: person, label
- Switch: 

"""

'\nGraph details:\n#1 Person-based (n = nPerson12*nBio8 = 96)\n  Person 1\nBlock 1: time (x) vs HR (left y) vs label (right y)\nBlock 2: time (x) vs Sys (left y) vs label (right y)\n...\nPerson 2\n=====\n#2 Label-based (n = nLabel5*nBio8 = 40)\nLabel 1: Time(x) vs Bio1 (Person 1-n :shape-color y)\nLabel 1: Time(x) vs Bio2 (Person 1-n :shape-color y)\n...\nLabel 0.75: Time(x) vs Bio2 (Person 1-n :shape-color y)\n...\n\nDaily = [\'ACTIVITY_TIME\',  \'CALORIES\', \'DISTANCE\',  \'STEP\', \'SLEEP\',]  \nI have much data with these columns:\n["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp", "Code", "Time (WIB)", "Name", "Device Code"]\n\nFirstly, merge the Name & Device Code.\nSecondly, create graph with Python\'s Plotly for each people (1 people 1 graph for each biomarker), the x-axis is the Time WIB, with vertical time label, the newest time in the right side. The first y-axis in the left is the biomarker columns (the column mentioned before without code, time, name, & devi

In [3]:
# Biomarker-based interactive dropdown
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

biomarkers = ["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp"]
labels = [1, 0.75, 0.5, 0.25, 0]
markers = ["circle", "circle-open", "cross-thin-open", "x-open", "x"]

# Read the data
file_label = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Label.csv"
df_raw = pd.read_csv(file_label)

# Merge Name and Device Code
df_raw['Name_Device'] = df_raw['Name'] + ' ' + df_raw['Device Code']
df_raw["Time (WIB)"] = pd.to_datetime(df_raw["Time (WIB)"])

# Get unique names for grouping
unique_names = pd.Index(df_raw['Name_Device'].unique())

app.layout = html.Div([
    html.Div([
        html.Label("Select Biomarker:"),
        dcc.Dropdown(
            options=[{'label': biomarker, 'value': biomarker} for biomarker in biomarkers],
            value=biomarkers[0],
            id='biomarker-dropdown'
        ),
        # html.Label("Select Labels:"),
        dcc.Checklist(
            options=[{'label': f'Label {label}', 'value': label} for label in labels],
            value=labels,
            id='label-checkbox',
            inline=True
        ),
        # html.Label("Select People:"),
        dcc.Checklist(
            options=[{'label': person.split()[0], 'value': person} for person in unique_names],
            value=unique_names,
            id='person-checkbox',
            inline=True,
            style={'columnCount': 6, 'width': '50000px'}
        )
    ], style={'width': '90%', 'margin': '0 auto', 'padding': '1px', 'color': 'black'}),

    dcc.Graph(id='biomarker-graph')
], style={'backgroundColor': '#F4F6F8'})


@callback(
    Output('biomarker-graph', 'figure'),
    Input('biomarker-dropdown', 'value'),
    Input('label-checkbox', 'value'),
    Input('person-checkbox', 'value')
)
def update_graph(biomarker, selected_labels, selected_people):
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, subplot_titles=[biomarker])

    for person in selected_people:
        df = df_raw[df_raw['Name_Device'] == person]

        for label in selected_labels:
            _df = df[df['Code'] == label]
            symbol = markers[labels.index(label)]

            fig.add_trace(go.Scatter(
                x=_df["Time (WIB)"],
                y=_df[biomarker],
                mode='markers',
                name=f"{person}-{label}",
                marker=dict(symbol=symbol, size=7)
            ))

    fig.update_layout(
        title=biomarker,
        xaxis_title='Time',
        yaxis_title='',
        height=600,
        showlegend=True
    )
    fig.update_yaxes(overlaying='y', side='right')
    return fig

if __name__ == '__main__':
    app.run(debug=True, host='localhost', port=8040)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AssertionError: 

In [1]:
# Biomarker-based interactive dropdown
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

biomarkers = ["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp"]
labels = [1, 0.75, 0.5, 0.25, 0]
markers = ["circle", "circle-open", "cross-thin-open", "x-open", "x"]

# Read the data
file_label  = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Label.csv"
df_raw = pd.read_csv(file_label)

# Merge Name and Device Code
df_raw['Name_Device'] = df_raw['Name'] + ' ' + df_raw['Device Code']
df_raw["Time (WIB)"] = pd.to_datetime(df_raw["Time (WIB)"])

# Get unique names for grouping
unique_names = pd.Index(df_raw['Name_Device'].unique())

"""
fig = make_subplots(rows=len(biomarkers), cols=1, shared_xaxes=True, subplot_titles=tuple(biomarkers), 
row_heights=[4 for _ in range(len(biomarkers))], vertical_spacing=0.01)

fig.add_trace(go.Scatter(x=_df["Time (WIB)"], y=_df[biomarker], mode='markers', name=f"{person}-{label}", marker=dict(symbol=symbol, size=7)), row=biomarkers.index(biomarker)+1, col=1)
    
fig.update_layout(title=biomarker, xaxis_title='Time', yaxis_title='', height=4000)
fig.update_yaxes(overlaying='y', side='right')

fig.write_html(f'visualization/biomarker-based.html')
"""

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    # html.Div([

        # html.Div([
            # dcc.Dropdown(
            #     df['Indicator Name'].unique(),
            #     'Fertility rate, xtotal (births per woman)',
            #     id='crossfilter-xaxis-column',
            # ),
            # dcc.RadioItems(
            #     ['Linear', 'Log'],
            #     'Linear',
            #     id='crossfilter-xaxis-type',
            #     labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            # )
        # ],
        # style={'width': '49%', 'display': 'inline-block'}),

        # html.Div([
        #     dcc.Dropdown(
        #         df['Indicator Name'].unique(),
        #         'Life expectancy at birth, total (years)',
        #         id='crossfilter-yaxis-column'
        #     ),
        #     dcc.RadioItems(
        #         ['Linear', 'Log'],
        #         'Linear',
        #         id='crossfilter-yaxis-type',
        #         labelStyle={'display': 'inline-block', 'marginTop': '5px'}
        #     )
        # ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    # ], style={
    #     'padding': '10px 5px'
    # }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):
    fig = px.scatter(dff, x='Year', y='Value')
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')
    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)
    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

if __name__ == '__main__':
    app.run(debug=True, host='localhost', port=8050)

/home/ihsannh17/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [7]:
# Biomarker based
# For every biomarker, plot the personal data vs the label (Oo-xX)

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

biomarkers = ["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp"]
labels = [1, 0.75, 0.5, 0.25, 0]
markers = ["circle", "circle-open", "cross-thin-open", "x-open", "x"]

# Read the data
file_label  = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Label.csv"
df_raw = pd.read_csv(file_label)

# Merge Name and Device Code
df_raw['Name_Device'] = df_raw['Name'] + ' ' + df_raw['Device Code']
df_raw["Time (WIB)"] = pd.to_datetime(df_raw["Time (WIB)"])

# Get unique names for grouping
unique_names = pd.Index(df_raw['Name_Device'].unique())

fig = make_subplots(rows=len(biomarkers), cols=1, shared_xaxes=True, subplot_titles=tuple(biomarkers), 
row_heights=[4 for _ in range(len(biomarkers))], vertical_spacing=0.01)

for biomarker in biomarkers:    
    for person in unique_names:
        df = df_raw[df_raw['Name_Device'] == person]

        for label in labels:
            _df = df[df['Code'] == label]
            symbol = markers[labels.index(label)]

            fig.add_trace(go.Scatter(x=_df["Time (WIB)"], y=_df[biomarker], mode='markers', name=f"{person}-{label}", marker=dict(symbol=symbol, size=7)), row=biomarkers.index(biomarker)+1, col=1)
    
    fig.update_layout(title=biomarker, xaxis_title='Time', yaxis_title='', height=4000)
    fig.update_yaxes(overlaying='y', side='right')

fig.write_html(f'visualization/biomarker-based.html')

In [2]:
# Label based # Done, IA, lillahilhamd
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

biomarkers = ["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp"]
labels = [1, 0.75, 0.5, 0.25, 0]
# Read the data
file_label  = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Label.csv"
df_raw = pd.read_csv(file_label)

# Merge Name and Device Code
df_raw['Name_Device'] = df_raw['Name'] + ' ' + df_raw['Device Code']
df_raw["Time (WIB)"] = pd.to_datetime(df_raw["Time (WIB)"])

# Get unique names for grouping
unique_names = pd.Index(df_raw['Name_Device'].unique())

for biomarker in biomarkers:
    fig = make_subplots(rows=len(labels), cols=1, shared_xaxes=True, subplot_titles=tuple(labels), row_heights=[4 for _ in range(len(labels))], vertical_spacing=0.01)
    
    for label in labels:
        df = df_raw[df_raw['Code'] == label]
        
        for person in unique_names:
            _df = df_raw[df_raw['Name_Device'] == person]
            fig.add_trace(go.Scatter(x=_df["Time (WIB)"], y=_df[biomarker], mode='markers', name=f"{person}-{label}", marker=dict(size=12)), row=labels.index(label)+1, col=1)
            
    
    fig.update_layout(title=biomarker, xaxis_title='Time', yaxis_title='Scaled Values', height=4000)
    fig.update_yaxes(overlaying='y', side='right')

    fig.write_html(f'visualization/label-based/biomarker_{list(biomarkers).index(biomarker)}.html')

In [4]:
# Person based (Enough, IA)
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

biomarkers = ["SpO2", "BP Dia", "BP Sys", "HRV", "Stress", "HR", "Body Temp"]

# Read the data
file_person = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Person.csv" 
df_raw = pd.read_csv(file_person)

# Merge Name and Device Code
df_raw['Name_Device'] = df_raw['Name'] + ' ' + df_raw['Device Code']

# Get unique names for grouping
unique_names = df_raw['Name_Device'].unique()

for person in unique_names:
    df = df_raw[df_raw['Name_Device'] == person]

    fig = make_subplots(rows=len(biomarkers), cols=1, shared_xaxes=True, subplot_titles=tuple(biomarkers), row_heights=[4 for _ in range(len(biomarkers))], vertical_spacing=0.01)

    for biomarker in biomarkers:
        scaled_label_df = df['Code']*(df[biomarker].max()-df[biomarker].min())+df[biomarker].min()

        fig.add_trace(go.Scatter(x=df.index, y=scaled_label_df, mode='lines', name=biomarker, marker=dict(size=8)), row=biomarkers.index(biomarker)+1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df[biomarker], mode='markers', name=biomarker, marker=dict(size=8)), row=biomarkers.index(biomarker)+1, col=1)

    # Update layout and display plots
    fig.update_layout(title=person, xaxis_title='Time', yaxis_title='Scaled Values', height=4000)
    fig.update_yaxes(overlaying='y', side='right')

    fig.write_html(f'visualization/person-based/biomarker_{list(unique_names).index(person)}.html')

In [ ]:
# Label-based
import pandas as pd
import plotly.express as px

# Load data from files A.csv and B.csv
file_person = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Person.csv" 
file_label  = "/home/ihsannh17/Downloads/RMP/github-repos/etc/2023-12-(04-14)_Label.csv"
data_A = pd.read_csv(file_person)
data_B = pd.read_csv(file_label)

# Merge 'Name' and 'Device Code' columns, ignore 'Event'
data_A['Person'] = data_A['Name'] + ' ' + data_A['Device Code']
data_A.drop(columns=['Name', 'Device Code', 'Event'], inplace=True)

data_B['Person'] = data_B['Name'] + ' ' + data_B['Device Code']
data_B.drop(columns=['Name', 'Device Code', 'Event'], inplace=True)

# Plot for Person-sorted data
fig_person = px.scatter(data_A, x='Time (WIB)', y='BP Dia', color='Person', facet_col='Code',
                        facet_col_wrap=3, title='Person-sorted data')
fig_person.update_xaxes(matches='x')
fig_person.update_layout(height=800, xaxis=dict(title='Timestamp (WIB)', tickangle=-90))

# Plot for Label-sorted data
fig_label = px.scatter(data_B, x='Time (WIB)', y='BP Dia', color='Person', facet_row='Code', facet_col='Code',
                       facet_col_wrap=3, title='Label-sorted data')
fig_label.update_xaxes(matches='x')
fig_label.update_layout(height=1200, xaxis=dict(title='Timestamp (WIB)', tickangle=-90))

# Display the plots
fig_person.show()
fig_label.show()


In [ ]:
# Activeness Detection by Temperature
import os
import pandas as pd

TEMPERATURE_THRESHOLD_LOW = 35.8 # degC
TEMPERATURE_THRESHOLD_HIGH = 40 # degC
TIME_THRESHOLD = 30 # min

folder_name = "Recap-2023-09-21"

folder_dir = os.getcwd()
folder_dir = os.path.join(folder_dir,folder_name)
files = os.listdir(folder_dir)
files.remove("recap.csv")

df_total = pd.DataFrame({'Tanggal':[],'Nama':[], 'Alat':[],'Aktif':[],'Shift':[]})
shift_bin = [0,5,11,17,23]
shift_label = [1,2,3,4]

files = [file for file in files if file.lower().endswith('.csv')]

for file in files:
    # DF Preparation
    device, name = file.split(" - ")[1:3]
    name = name[:-4]

    file_dir = os.path.join(folder_dir,file)
    condition1 = os.stat(file_dir).st_size == 0
    condition2 = " TTL" not in file.split("-")
    condition3 = name == "noname"
    condition4 = not(name.isupper())
    skip_criteria = [condition1, condition2, condition3, condition4]
    if (True in skip_criteria): continue
    df = pd.read_csv(file_dir)

    try:
        df_person = pd.DataFrame({
            'time': pd.to_datetime(df['time']),
            'T' : df['TEMPERATURE']
        })
    except KeyError: continue

    df_person = df_person.sort_values(by='time',ascending=False).query('@TEMPERATURE_THRESHOLD_LOW < T < @TEMPERATURE_THRESHOLD_HIGH')
    df_person['Aktif'] = (df_person['time'] - df_person['time'].shift(-1)).dt.total_seconds()/60
    df_person = df_person.query('Aktif <= 30')
    df_person['Tanggal'] = df_person['time'].dt.date
    df_person['Jam'] = df_person['time'].dt.hour
    df_person['Shift'] = pd.cut(df_person['Jam'], bins=shift_bin, labels=shift_label, right=False)
    _shift = df_person.groupby(['Tanggal', 'Shift']).size().reset_index(name='Jumlah')
    shift = _shift.loc[_shift.groupby('Tanggal')['Jumlah'].idxmax()]

    df_recap = round((df_person.groupby('Tanggal')['Aktif'].sum()/60),1).reset_index()
    df_recap['Nama'] = [f'{name}']*len(df_recap)
    df_recap['Alat'] = [f'{device}']*len(df_recap)
    df_recap = pd.merge(df_recap, shift, on='Tanggal').drop(columns='Jumlah')
    df_recap = df_recap[['Tanggal','Nama', 'Alat','Aktif','Shift']].sort_values(by='Tanggal', ascending=False)
    
    df_total = pd.concat([df_total, df_recap], ignore_index=True).sort_values(by=['Tanggal','Nama'], ascending=False)

date_min = df_total['Tanggal'].min()
date_max = df_total['Tanggal'].max()
df_total.to_excel(f'Keaktifan_Covwatch_TTL_{date_min}_{date_max}.xlsx',index=False)

### Penggabungan File Recap dari Bot Telegram

In [ ]:
# Merge with sort according to name > device > date
import os
import pandas as pd

# membuat file csv sumber
columns = ['ACTIVITY_TIME', 'BATTERY', 'BLOOD_DIASTOLIC', 'BLOOD_SYSTOLIC',
           'CALORIES', 'DISTANCE', 'HEART_RATE', 'HRV', 'SLEEP',
           'SPO2', 'STEP', 'STRESS', 'TEMPERATURE', 'time', 'TANGGAL','JAM','NAMA_FILE',
           'KODE_DEVICE', 'KODE_DB', 'KATEGORI_WAKTU', 'SHIFT_PEGAWAI']

# Membuat DataFrame kosong berdasarkan nama kolom yang ditentukan
df_sumber = pd.DataFrame(columns=columns)
df_sumber.to_csv("./sumber.csv",index=False)

#mengabungkan file -  file csv pada setiap folder
df_sumber = pd.read_csv("./sumber.csv")
kolom = ['dummy_kolom']
baris = ["\n"]
folder_target = "Recap-2023-11-22"
folder_asli = list(os.listdir(f"./{folder_target}"))
folder_asli.remove("recap.csv")

#membuka setiap file pada folder
def sorting_key(filename):
    try:
        db, company, name = filename.split(" - ")
        return (name, company, db)
    except:
        return (f"{filename}","Error","-")

try:
    folder_urut = sorted(folder_asli,key=sorting_key)
except TypeError:
    pass

for filename in folder_urut:
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_target, filename)
        condition1 = os.stat(file_path).st_size == 0
        condition2 = " TTL" in filename.split("-")
        condition3 = " RSA" in filename.split("-")
        if (condition1 or not (condition2 or condition3)):
            continue
        with open(file_path) as f:
            first_line = f.readline().strip()
            if not first_line:
                continue
                     
        df2 = pd.read_csv(file_path)
        #membuat kolom nama, kode device, kode database
        filename = filename.split(" - ")
        df2['NAMA_FILE'] = (filename[2])
        df2['KODE_DEVICE'] = (filename[1])
        df2['KODE_DB'] = (filename[0])

        #membuat kategori shift sesuai waktu 
        if "time" in df2.columns.tolist():
            df2['time'] = pd.to_datetime(df2['time'],format='%Y-%m-%d %H:%M')
            def kategori_waktu(waktu):
                if waktu.hour < 6:
                    return 'shift 1'
                elif waktu.hour < 12:
                    return 'shift 2'
                elif waktu.hour < 18:
                    return 'shift 3'
                else:
                    return 'shift 4'
            df2['KATEGORI_WAKTU'] = df2['time'].apply(kategori_waktu)

            # Menghitung jumlah kategori shift yang paling sering muncul dari seluruh tabel
            kategori_terbanyak = df2['KATEGORI_WAKTU'].value_counts().idxmax()

            # Membuat kolom baru 'Kategori Terbanyak' dan mengisikan nilai kategori terbanyak ke setiap baris
            df2['SHIFT_PEGAWAI'] = kategori_terbanyak

            #membagi tanggal dan jam menjadi kolom tersendiri
            df2['time'] = pd.to_datetime(df2['time'])
            df2["TANGGAL"] = df2['time'].dt.date
            df2["JAM"] = df2['time'].dt.time
            #mengurutkan data berdasarkan waktu
            df2 = df2.sort_values(by='time')

            empty_row = pd.Series({'time': ''})
            df2 = df2.append(empty_row, ignore_index=True)
        #menggabungkan setiap file csv kedalam satu file csv sumber
        df_sumber = pd.concat([df_sumber, df2], ignore_index=True)
                     
#menyimpan file csv kedalam file
df_sumber.to_csv(f"./TTL-RSA-{folder_target}.csv", index=False)



In [1]:
# Merge Didit (adapted)
import os
import pandas as pd

# membuat file csv sumber
import pandas as pd
columns = ['ACTIVITY_TIME', 'BATTERY', 'BLOOD_DIASTOLIC', 'BLOOD_SYSTOLIC',
           'CALORIES', 'DISTANCE', 'HEART_RATE', 'HRV', 'SLEEP',
           'SPO2', 'STEP', 'STRESS', 'TEMPERATURE', 'time', 'TANGGAL','JAM','NAMA_FILE',
           'KODE_DEVICE', 'KODE_DB', 'KATEGORI_WAKTU', 'SHIFT_PEGAWAI']

# Membuat DataFrame kosong berdasarkan nama kolom yang ditentukan
df_sumber = pd.DataFrame(columns=columns)
df_sumber.to_csv("./sumber.csv",index=False)

#mengabungkan file -  file csv pada setiap folder
df_sumber = pd.read_csv("./sumber.csv")
kolom = ['dummy_kolom']
baris = ["\n"]
folder_target = "Recap-2023-12-14"
folder_asli = list(os.listdir(f"./{folder_target}"))
folder_asli.remove("recap.csv")

#membuka setiap file pada folder
def sorting_key(filename):
    try:
        db, company, name = filename.split(" - ")
        return (name, company, db)
    except:
        return (f"{filename}","Error","-")

try:
    folder_urut = sorted(folder_asli,key=sorting_key)
except TypeError:
    pass

for filename in folder_urut:
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_target, filename)
        condition1 = os.stat(file_path).st_size == 0
        condition2 = " TTL" in filename.split("-")
        condition3 = " RSA" in filename.split("-")
        if (condition1 or not (condition2 or condition3)):
            continue
        with open(file_path) as f:
            first_line = f.readline().strip()
            if not first_line:
                continue
                     
        df2 = pd.read_csv(file_path)
        #membuat kolom nama, kode device, kode database
        filename = filename.split(" - ")
        df2['NAMA_FILE'] = (filename[2])
        df2['KODE_DEVICE'] = (filename[1])
        df2['KODE_DB'] = (filename[0])

        #membuat kategori shift sesuai waktu 
        if "time" in df2.columns.tolist():
            df2['time'] = pd.to_datetime(df2['time'],format='%Y-%m-%d %H:%M')
            def kategori_waktu(waktu):
                if waktu.hour < 6:
                    return 'shift 1'
                elif waktu.hour < 12:
                    return 'shift 2'
                elif waktu.hour < 18:
                    return 'shift 3'
                else:
                    return 'shift 4'
            df2['KATEGORI_WAKTU'] = df2['time'].apply(kategori_waktu)

            # Menghitung jumlah kategori shift yang paling sering muncul dari seluruh tabel
            kategori_terbanyak = df2['KATEGORI_WAKTU'].value_counts().idxmax()

            # Membuat kolom baru 'Kategori Terbanyak' dan mengisikan nilai kategori terbanyak ke setiap baris
            df2['SHIFT_PEGAWAI'] = kategori_terbanyak

            #membagi tanggal dan jam menjadi kolom tersendiri
            df2['time'] = pd.to_datetime(df2['time'])
            df2["TANGGAL"] = df2['time'].dt.date
            df2["JAM"] = df2['time'].dt.time
            #mengurutkan data berdasarkan waktu
            df2 = df2.sort_values(by='time')

            empty_row = pd.Series({'time': ''})
            df2 = df2.append(empty_row, ignore_index=True)
        #menggabungkan setiap file csv kedalam satu file csv sumber
        df_sumber = pd.concat([df_sumber, df2], ignore_index=True)
                     
#menyimpan file csv kedalam file
df_sumber.to_csv(f"./TTL-RSA-{folder_target}.csv", index=False)



/tmp/ipykernel_32334/325036755.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(empty_row, ignore_index=True)
/tmp/ipykernel_32334/325036755.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(empty_row, ignore_index=True)
/tmp/ipykernel_32334/325036755.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(empty_row, ignore_index=True)
/tmp/ipykernel_32334/325036755.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(empty_row, ignore_index=True)
/tmp/ipykernel_32334/325036755.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pand

### Analisis Marker


In [3]:
# V2

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read CSV file into a DataFrame
filedir = "/home/ihsannh17/Downloads/RMP/github-repos/etc/TTL-Recap-230104-5.csv"
df = pd.read_csv(filedir)

# Plot for Battery, BP Dia, BP Sys, HR, HRV, SpO2, Stress, Body Temp, Activity Code
fig1 = make_subplots(rows=1, cols=1, shared_xaxes=True, subplot_titles=('First Subplot'))
for col in ['Battery', 'BP Dia', 'BP Sys', 'HR', 'HRV', 'SpO2', 'Stress', 'Body Temp', 'Activity Code']:
    fig1.add_trace(go.Scatter(x=df['Time'], y=df[col], mode='lines', name=col))

fig1.update_layout(title='Metrics Plot - Part 1', xaxis_title='Time', yaxis_title='Values')

# Plot for Step, Calories, Distance, Activity Time
fig2 = make_subplots(rows=1, cols=1, shared_xaxes=True, subplot_titles=('Second Subplot'))
for col in ['Step', 'Calories', 'Distance', 'Activity Time']:
    fig2.add_trace(go.Scatter(x=df['Time'], y=df[col], mode='lines', name=col))

fig2.update_layout(title='Metrics Plot - Part 2', xaxis_title='Time', yaxis_title='Values')

# Show plots
fig1.show()
fig2.show()


In [5]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read CSV file into a DataFrame
filedir = "/home/ihsannh17/Downloads/RMP/github-repos/etc/TTL-Recap-230104-5.csv"
df = pd.read_csv(filedir)

# Select numeric columns for scaling
numeric_cols = df.select_dtypes(include='number').columns

# Filter out non-numeric columns
numeric_df = df[numeric_cols]

# Scale the numeric data
scaled_df = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())

# Create subplots with different heights for the two plots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('First Subplot', 'Second Subplot'), row_heights=[0.7, 0.3])

# Plot scaled values for the first set of parameters
for col in ['Battery', 'BP Dia', 'BP Sys', 'HR', 'HRV', 'SpO2', 'Stress', 'Body Temp', 'Activity Code']:
    fig.add_trace(go.Scatter(x=df.index, y=scaled_df[col], mode='markers', name=col, marker=dict(size=8)), row=1, col=1)

# Plot scaled values for the second set of parameters
for col in ['Step', 'Calories', 'Distance', 'Activity Time']:
    fig.add_trace(go.Scatter(x=df.index, y=scaled_df[col], mode='markers', name=col, marker=dict(size=8)), row=2, col=1)

# Update layout and display plots
fig.update_layout(title='Scaled Metrics Plot', xaxis_title='Index', yaxis_title='Scaled Values', height=800)
fig.update_yaxes(overlaying='y', side='right')

fig.show()

In [20]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read CSV file into a DataFrame
filedir = "/home/ihsannh17/Downloads/RMP/github-repos/etc/TTL-Recap-230104-5-SortedActivity.csv"
df = pd.read_csv(filedir)

# Select numeric columns for scaling
numeric_cols = df.select_dtypes(include='number').columns

# Filter out non-numeric columns
numeric_df = df[numeric_cols]

# Scale the numeric data
scaled_df = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())

# Create subplots with different heights for the two plots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('First Subplot', 'Second Subplot'), row_heights=[0.7, 0.3])

# Plot scaled values for the first set of parameters
buttons_1 = []
for col in ['Battery', 'BP Dia', 'BP Sys', 'HR', 'HRV', 'SpO2', 'Stress', 'Body Temp', 'Activity Code']:
    scatter = go.Scatter(x=df.index, y=scaled_df[col], mode='markers', name=col, marker=dict(size=8), visible='legendonly')
    fig.add_trace(scatter, row=1, col=1)
    buttons_1.append(dict(label=col, method="update", args=[{"visible": [col == trace.name or trace.visible == 'legendonly' for trace in fig.data]}]))

# Plot scaled values for the second set of parameters
buttons_2 = []
for col in ['Step', 'Calories', 'Distance', 'Activity Time']:
    scatter = go.Scatter(x=df.index, y=scaled_df[col], mode='markers', name=col, marker=dict(size=8), visible='legendonly')
    fig.add_trace(scatter, row=2, col=1)
    buttons_2.append(dict(label=col, method="update", args=[{"visible": [col == trace.name or trace.visible == 'legendonly' for trace in fig.data]}]))

# Update layout and display plots
fig.update_layout(
    title='Scaled Metrics Plot',
    xaxis_title='Index',
    yaxis_title='Scaled Values',
    height=800,
    updatemenus=[
        dict(buttons=buttons_1, direction="down", pad={"r": 10, "t": 10}, showactive=True, x=0.01, xanchor="left", y=1.1, yanchor="top"),
        dict(buttons=buttons_2, direction="down", pad={"r": 10, "t": 10}, showactive=True, x=0.01, xanchor="left", y=0.45, yanchor="top")
    ]
)

fig.show()

In [46]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

folder_name = "Recap-2023-09-14"

folder_dir = os.getcwd()
folder_dir = os.path.join(folder_dir,folder_name)
files = os.listdir(folder_dir)
files.remove("recap.csv")

files = [file for file in files if file.lower().endswith('.csv')]

hold = 0
trace = {}

for file in files:
    # DF Preparation
    device, name = file.split(" - ")[1:3]
    name = name[:-4]

    file_dir = os.path.join(folder_dir,file)
    condition1 = os.stat(file_dir).st_size == 0
    condition2 = " TTL" not in file.split("-")
    condition3 = name == "noname"
    condition4 = not(name.isupper())
    skip_criteria = [condition1, condition2, condition3, condition4]
    if (True in skip_criteria): continue
    df = pd.read_csv(file_dir)
    df["time"] = pd.to_datetime(df["time"])
    compared_columns = list(df.columns)
    
    if hold < 10e8:
        trace['TEMPERATURE'] = go.Scatter(x=df["time"],y=df["TEMPERATURE"],mode='markers', name='Temperature')
        for col in [col for col in df.columns if col not in ['TEMPERATURE','time']] :
            trace[f"{col}"] = go.Scatter(x=df['time'], y=df[f'{col}'], mode = 'markers', name=f'{col}', yaxis='y2')

            layout = go.Layout(
                title = f't-T-{col}',
                xaxis = dict(title='Time'),
                yaxis = dict(title='Temperature'),
                yaxis2 = dict(title=f'{col}',overlaying='y', side='right', rangemode='tozero'),
                height=600
            )
            fig = go.Figure(data=[trace['TEMPERATURE'],trace[f"{col}"]],layout=layout)
            fig.update_xaxes(rangeslider_visible=True)
            if col == "HEART_RATE": fig.show()
        # break

    

/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/ihsannh17/miniconda3/envs/cling/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



KeyError: 'TEMPERATURE'

In [31]:
list(df.columns)

['ACTIVITY_TIME',
 'BATTERY',
 'BLOOD_DIASTOLIC',
 'BLOOD_SYSTOLIC',
 'CALORIES',
 'DISTANCE',
 'HEART_RATE',
 'HRV',
 'SPO2',
 'STEP',
 'STRESS',
 'TEMPERATURE',
 'time']

In [5]:
import plotly.graph_objects as go
import pandas as pd

# Sample data
data = {'Date': pd.date_range('2023-01-01', periods=100),
        'Value1': [i for i in range(100)],
        'Value2': [i*2 for i in range(100)]}

df = pd.DataFrame(data)

# Create traces
trace1 = go.Scatter(x=df['Date'], y=df['Value1'], mode='lines', name='Value1')
trace2 = go.Scatter(x=df['Date'], y=df['Value2'], mode='lines', name='Value2', yaxis='y2')

# Define layout
layout = go.Layout(
    title='Dual Y-Axis Example',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Value1', rangemode='tozero'),
    yaxis2=dict(title='Value2', overlaying='y', side='right', rangemode='tozero')
)

# Create figure
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)
)

# Show the plot
fig.show()


In [36]:
a = [1,2,3].remove(2)
print(a)

None
